In [1]:
import os
from PIL import Image as Img
import numpy as np
import cv2
import pickle

from keras_facenet import FaceNet



In [2]:
MyFaceNet = FaceNet()

In [3]:
HaarCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:

# Path ke folder data sampel
folder = "sample_data"
database = {}

# Proses untuk memuat data sampel dan membuat database
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    gbr1 = cv2.imread(path)

    # Mengonversi gambar ke skala abu-abu untuk deteksi wajah
    gray = cv2.cvtColor(gbr1, cv2.COLOR_BGR2GRAY)

    # Mendeteksi wajah
    wajah = HaarCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    if len(wajah) > 0:
        x1, y1, width, height = wajah[0]
    else:
        x1, y1, width, height = 1, 1, 10, 10

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Img.fromarray(gbr)
    # Konversi dari OpenCV ke PIL
    gbr_array = np.asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]

    face = Img.fromarray(face)
    face = face.resize((160, 160))
    face = np.asarray(face)

    face = np.expand_dims(face, axis=0)

    # Mengasumsikan MyFaceNet telah didefinisikan dan dimuat
    signature = MyFaceNet.embeddings(face)

    database[os.path.splitext(filename)[0]] = signature

1/1 [==============================] - 0s 335ms/step


In [5]:

# Menyimpan database ke file
with open("dataset.pkl", "wb") as file:
    pickle.dump(database, file)

In [6]:

# Membuka kembali file database
with open("dataset.pkl", "rb") as file:
    database = pickle.load(file)

In [7]:
def findFaces(data):
    gbr1 = data
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = np.asarray(gbr)

    wajah = HaarCascade.detectMultiScale(gbr1, 1.1, 4)

    identities = []  # Simpan semua identitas yang ditemukan dalam satu frame

    for (x1, y1, w, h) in wajah:
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + w, y1 + h

        face = gbr_array[y1:y2, x1:x2]

        face = Image.fromarray(face)
        face = face.resize((160, 160))
        face = np.asarray(face)


        face = np.expand_dims(face, axis=0)
        signature = MyFaceNet.embeddings(face)
        min_dist = float('inf')
        identity = 'Tidak Dikenal'  # Default identity jika tidak ada yang cocok

        for key, value in database.items():
            dist = np.linalg.norm(value - signature)
            if dist < min_dist:
                min_dist = dist
                identity = key

        # Hapus angka dari identitas
        cleaned_identity = ''.join(filter(str.isalpha, identity))

        # Tambahkan identitas ke daftar
        identities.append(cleaned_identity)

        cv2.putText(gbr1, cleaned_identity, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA)
        cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)

    filename = 'photo.jpg'
    cv2.imwrite(filename, gbr1)

    # Cetak nama identitas ke console
    print("Identities found:", identities)

    return filename

In [12]:
from PIL import Image
def take_photo():
    URLS = 'http://192.168.13.112:8080///video'

    cap = cv2.VideoCapture(URLS)
    ret, frame = cap.read()
    cap.release()

    filename = findFaces(frame)

In [16]:

try:
    take_photo()
except Exception as err:
    print(str(err))

1/1 [==============================] - 0s 454ms/step
Identities found: ['carlos', 'VickyArdiansyah']
